In [10]:
import librosa
import numpy as np
import IPython.display as ipd
import os
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

## Preparing data

#### Preparing start samples

In [14]:
X_start = []
paths = librosa.util.find_files(r'augmented')
for file in paths:
    x, sr = librosa.load(file)
    mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=len(x)//64, n_mfcc=20)
    X_start.append(mfcc)
X_start = np.array(X_start)
X_start = X_start.reshape(*X_start.shape, 1)
y_start = np.ones(np.shape(X_start)[0])
np.shape(X_start)

(426, 20, 65, 1)

#### Preparing other samples

In [37]:
def kernel(folder):
    tmp = []
    with os.scandir(folder) as entries:
        for entry in entries:
            if entry.is_file():
                x, sr = librosa.load(entry)
                mfcc = librosa.feature.mfcc(y=x, sr=sr, hop_length=len(x)//64, n_mfcc=20)
                tmp.append(mfcc)
    return tmp

voices_path = r'audiobooks\data\ru_open_stt\private_buriy_audiobooks_2\0'
folders = list(os.scandir(voices_path))
pool = ThreadPool(len(folders))
X_other = pool.map(kernel, folders)
pool.close()
pool.join()

X_other = [j for i in np.arange(len(X_other)) for j in X_other[i]]
X_other = np.array(X_other).reshape(*np.shape(X_other), 1)
y_other = np.zeros(np.shape(X_other)[0])
np.shape(X_other)

Wall time: 1min 42s


## Loading data

#### Loading start samples

In [24]:
data = np.load('data/start.npz')
X_start = data['sample']
y_start = data['label']
X_start.shape

(426, 20, 65, 1)

#### Loading other samples

In [25]:
data = np.load('data/other.npz')
X_other = data['sample']
y_other = data['label']
X_other.shape

(3359, 20, 65, 1)

## Merging

In [26]:
X = np.append(X_start, X_other, axis=0)
Y = np.append(y_start, y_other, axis=0)
X.shape

(3785, 20, 65, 1)

## Saving samples

#### Saving start samples

In [20]:
np.savez_compressed('data/start', sample=X_start, label=y_start)

#### Saving other samples

In [21]:
np.savez_compressed('data/other', sample=X_other, label=y_other)

#### Saving merged samples

In [22]:
np.savez_compressed('data/merged', sample=X, label=Y)